In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from keras.preprocessing import image

img1 = image.load_img("../input/celeba-dataset/img_align_celeba/img_align_celeba/000001.jpg", target_size = (128, 128))
img1_arr = image.img_to_array(img1, dtype = 'int64')
img1_arr = np.asarray(img1_arr)

In [ ]:
img1_arr

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(img1_arr)
plt.show()

In [ ]:
# Load all the images from the dataset

path = "../input/celeba-dataset/img_align_celeba/img_align_celeba"
def load_image_data(path):
    
    ''' Reads the images from the path and returns the images in to form of numpy array.'''
    
    img_folder = os.listdir(path)
    
    # Create an Image data list
    img_data = []
    
    # Iterate over each image load it using image.load_img and then convert it using img_to_array
    for img in img_folder[:5000]:
        
        # load the img
        ith_img = image.load_img(os.path.join(path,img), target_size = (128, 128))
        ith_img_arr = image.img_to_array(ith_img, dtype = 'int64')
        
        # append into the list
        img_data.append(ith_img_arr)
        
    return img_data

In [ ]:
len(os.listdir(path))

In [ ]:
images = load_image_data(path)

In [ ]:
# Plot first 10 images
plt.figure(figsize = (10, 10))
for i in range(10):
    
    plt.subplot(2, 5, i+1)
    plt.imshow(images[i])

In [ ]:
images[0].shape

In [ ]:
images_arr = np.asarray(images)

In [ ]:
images_arr = (images_arr.astype('float32') - 127.5)/127.5

In [ ]:
np.max(images_arr), np.min(images_arr)

In [ ]:
images_arr.shape[0]

# Building a DCGAN

In [ ]:
from keras.layers import *
from keras.models import Model, Sequential
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam

In [ ]:
epochs = 50
batch_size = 256
half_batch_size = 128
no_of_batches = int(images_arr.shape[0]/batch_size)
noise_dim = 100
adam = Adam(lr = 2e-4, beta_1 = 0.5)

In [ ]:
images_arr.shape

In [ ]:
# Define the Generator

generator = Sequential()
generator.add(Dense(16*16*128, input_shape = (noise_dim, )))
generator.add(Reshape((16,16,128)))
generator.add(LeakyReLU(0.2))
generator.add(BatchNormalization())

# Double the activation sie 32 X 32 X 64
generator.add(UpSampling2D())
generator.add(Conv2D(64, kernel_size = (5,5), padding = 'same'))
generator.add(LeakyReLU(0.2))
generator.add(BatchNormalization())

# Double the activation sie 64 X 64 X 32
generator.add(UpSampling2D())
generator.add(Conv2D(32, kernel_size = (5,5), padding = 'same'))
generator.add(LeakyReLU(0.2))
generator.add(BatchNormalization())

# Double the activation sie 128 X 128 X 3
generator.add(UpSampling2D())
generator.add(Conv2D(3,activation = 'tanh', kernel_size = (5,5), padding = 'same'))


# Compile the model
generator.compile(loss='binary_crossentropy', optimizer = adam)
generator.summary()


In [ ]:
# Build a Discriminator
discriminator = Sequential()

# Input layer with shape 128 X 128 X 3
discriminator.add(Conv2D(32, (5,5), strides = (2,2), padding = 'same', input_shape =(128,128,3)))
discriminator.add(LeakyReLU(0.2))

# Reduce the size from 64 X 64 X 32 to 32 X 32 X 64
discriminator.add(Conv2D(64, (5,5), strides = (2,2), padding = 'same'))
discriminator.add(LeakyReLU(0.2))

# Reduce the size further from 32 X 32 X 64 to 16 X 16 X 128
discriminator.add(Conv2D(128, (5,5), strides = (2,2), padding = 'same'))
discriminator.add(LeakyReLU(0.2))

# Flatten the layer
discriminator.add(Flatten())
discriminator.add(Dense(1, activation = 'sigmoid'))

# Compile the model
discriminator.compile(loss = 'binary_crossentropy', optimizer = adam)
discriminator.summary()


# Training the GANs

In [ ]:
discriminator.trainable = False
gan_input = Input(shape = (noise_dim,))
generated_img = generator(gan_input)
gan_output = discriminator(generated_img)

# Functional API
model = Model(gan_input, gan_output)
model.compile(loss = 'binary_crossentropy', optimizer = adam)


In [ ]:
def plot_imgs(epoch, samples = 100):
    
    noise = np.random.normal(0,1, size = (samples, noise_dim))
    generated_imgs = generator.predict(noise)
    generated_imgs = generated_imgs.reshape(samples, 128, 128, 3)
    
    # Plot the generated images
    plt.figure(figsize = (20, 20))
    for i in range(samples):
        
        plt.subplot(10, 10, i+1)
        plt.imshow(generated_imgs[i], interpolation = 'nearest')
        plt.axis("off")
        
    plt.tight_layout()
    plt.show()
    
    

In [ ]:
# Training Loop
for epoch in range(epochs):
    
    epoch_d_loss = 0.
    epoch_g_loss = 0.
    
    # Mini bathc SGD
    for step in range(no_of_batches):
        
        # Step 1 train discriminator: 50% real data + 50% Fake images
        
        # Real Data
        idx = np.random.randint(0, 5000, half_batch_size)
        real_imgs = images_arr[idx]
        
        # Fake Data
        noise = np.random.normal(0,1, size = (half_batch_size, noise_dim))
        fake_imgs = generator.predict(noise)
        
        # Labels
        real_y = np.ones((half_batch_size,1))*0.9
        fake_y = np.zeros((half_batch_size,1))
        
        # Train our discriminator
        d_real_loss = discriminator.train_on_batch(real_imgs, real_y)
        d_fake_loss = discriminator.train_on_batch(fake_imgs, fake_y)
        
        d_loss = 0.5*d_real_loss + 0.5*d_fake_loss
        
        epoch_d_loss += d_loss
        
        # Train the generator (Considering Generator as frozen)
        noise = np.random.normal(0,1, size = (batch_size, noise_dim))
        ground_truth_y = np.ones((batch_size, 1))
        
        g_loss = model.train_on_batch(noise, ground_truth_y)
        
        epoch_g_loss =+ g_loss
    
    print("Epoch %d Disc. loss %.4f Generator Loss %.4f"%((epoch+1), epoch_d_loss/no_of_batches, epoch_g_loss/no_of_batches))
    if (epoch + 1)%5 == 0:
        
        plot_imgs(epoch)